In [12]:
# Week 5 - Milestone 1: Smart Tracking System Blockchain Ledger (Submission)

# This script is intended to be run in a Jupyter Notebook environment.
# To run this script, ensure you have Jupyter Notebook installed and running.

# You can install Jupyter Notebook using pip:
# %pip install notebook

# You can start Jupyter Notebook by running the following command in your terminal:
# $ jupyter notebook 

# This script connects to a local Ganache blockchain using Web3.py.
# Make sure you have Ganache running and Web3.py installed.
# %pip install web3

from web3 import Web3
import json, time


# Load CSV file from your Homework 1
import pandas as pd


# Load IoT sensor data from CSV file (Generated in Homework 1)
df = pd.read_csv('../Week 5 Homework/iot_data.csv')

# Display the first few rows of the DataFrame
print(df.head())

# Connect to local Ganache blockchain
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# Check if connected to Ganache
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")
    exit()

# Replace with actual contract address form Remix 
contract_address = "0x7772e6a1cbfFf847250B44aaCb2290Ba9b6F1347"

# Paste the ABI from Remix 
# abi = [...] # Replace with the actual ABI of your contract

# If you have the ABI in a file, you can load it like this:
with open('copy-of-abi.json', 'r') as abi_file:
    abi = json.load(abi_file)

# Load the smart contract
contract = web3.eth.contract(address=contract_address, abi=abi)

# Set the default sender address (the first account from Ganache)
web3.eth.default_account = web3.eth.accounts[0]

print(f"✅ Connected to Smart Contract at {contract_address}")


# Send the IoT data to the blockchain. Each row from the CSV file will be stored as a transaction on the blockchain.
def send_iot_data(package_id, gps_coords, temperature, status):
    
    txn = contract.functions.storeData(
        package_id,
        gps_coords,
        temperature,
        status
    ).transact({
        'from': web3.eth.default_account,
        'gas': 3000000
    })
    
    receipt = web3.eth.wait_for_transaction_receipt(txn)
    print(f"✅ Data Stored {package_id} GPS: {gps_coords:>23} Temp: {temperature}  Status: {status:<10} at block {
        receipt.blockNumber:>3} Txn hash {receipt.transactionHash.hex()}")


# Iterate through the DataFrame and send each row to the blockchain
for index, row in df.iterrows():

    package_id  = str(row['package_id'  ])
    gps_coords  = str(row['gps_coords'  ])
    temperature = str(row['temperature' ])
    status      = str(row['status'      ])    

    send_iot_data(package_id, gps_coords, temperature, status)    
    time.sleep(1)  # sleep to avoid hitting the gas limit too quickly

# now that the data is on the blockchain, we can retrieve it to verify storage
total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

# retrieve and print a specific record (e.g., the first one)
record = contract.functions.getRecord(0).call()
print(f"First Stored Record : {record}")



               timestamp package_id               gps_coords temperature  \
0  2025-06-02T21:36:32I0     PKG001   (7.839114, 125.820784)        29 C   
1  2025-06-02T21:36:32I1     PKG002  (16.754497, 116.599905)        24 C   
2  2025-06-02T21:36:32I2     PKG003  (16.521005, 119.641554)        29 C   
3  2025-06-02T21:36:32I3     PKG004  (10.941567, 118.303819)        29 C   
4  2025-06-02T21:36:32I4     PKG005   (9.844815, 118.509812)        29 C   

       status  
0   Delivered  
1   Delivered  
2     Delayed  
3   Delivered  
4  In Transit  
✅ Connected to Ganache successfully!
✅ Connected to Smart Contract at 0x7772e6a1cbfFf847250B44aaCb2290Ba9b6F1347
✅ Data Stored PKG001 GPS:  (7.839114, 125.820784) Temp: 29 C  Status: Delivered  at block   2 Txn hash 19fbb7c4745279ecc3046057e692b3d007e9664fc8654283b7138bed5bc2a0a8
✅ Data Stored PKG002 GPS: (16.754497, 116.599905) Temp: 24 C  Status: Delivered  at block   3 Txn hash 0e714d61602737dd2a8134f37c294497dfb486bb63803169cd943200aa4cc6d